# [SEMES] - 볼트 이상진단 - 전이학습

#### 전이학습 - 학습을 진행할 때 필요한 라이브러리 import

In [ ]:
import os
# 파이토치의 핵심 패키지(모델 구성 및 학습 등을 수행할 수 있는 기능을 제공)
import torch
# PyTorch에서 제공하는 신경망 모듈
import torch.nn as nn
# 학습에 사용되는 최적화 알고리즘
import torch.optim as optim
# PyTorch에서 이미지 데이터 처리와 관련된 함수와 모델들을 제공
from torchvision import models, datasets
# transforms 모듈은 데이터 전처리를 위한 함수들을 제공
import torchvision.transforms as transforms
# DataLoader를 이용하여 데이터셋에서 미니배치(minibatch)를 추출 
from torch.utils.data import DataLoader
# 시간과 관련된 함수를 제공
import time


# 그래프 - 데이터 시각화를 위해 그래프나 도형을 화면에 출력해줌
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torchvision

# 컨퓨전 매트릭스 - sklearn.metrics == 평가 지표를 계산하는 패키지 
# 컨퓨전 매트릭스 - classification_report == 분류 모델을 평가한 후 각 클래스마다 지표를 출력(정확도(Accuracy)0/밀도(Precision)/재현율(Recall)/F1-점수(F1-Score)/지원 개수(Support)
# f1 score - f1_score == 분류 모델을 평가하는 지표- 정밀도(precision)와 재현율(recall)의 조화평균(정확도와 유사하지만 불균형한 데이터에서도 적절한지 평가할 수 있는 지표가 됨)
from sklearn.metrics import classification_report, f1_score

# 컨퓨전 매트릭스 - 데이터 시각화 라이브러리
import seaborn as sns

# 평가코드
from PIL import Image
import torch.nn.functional as F

#### 전이학습 - GPU 설정

In [ ]:
# GPU가 사용 가능한 경우 cuda 사용 / GPU가 사용 불가능한 경우 CPU로 초기화하여 CPU 사용
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using cuda")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU instead.")

#### 전이학습 - 데이터 전처리

In [ ]:
# 학습데이터 전처리
train_transform = transforms.Compose([
    # 해상도를 (224,224)로 맞춰준다 (a fixed resolution of 224×224 is best, even at higher flops : [논문]Designing Network Design Spaces - [저자]Facebook AI Research (FAIR))
    transforms.Resize((224, 224)),
    # 이미지를 좌우로 뒤집어서 데이터 증강(augmentation)을 수행(확률을 높여준)
    transforms.RandomHorizontalFlip(),
    # 이미지를 PyTorch의 Tensor로 변환
    transforms.ToTensor(),
    # 흑백 이미지이기 때문에 1개의 채널을 정규화(흑백이미지는 보통 (평균 : 0.5 / 표준편차 : 0.5)로 정규화)
    transforms.Normalize([0.5], [0.5])
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

])

# 테스트데이터 전처리
test_transform = transforms.Compose([
    # 해상도를 (224,224)로 맞춰준다
    transforms.Resize((224, 224)),
    # 이미지를 PyTorch의 Tensor로 변환
    transforms.ToTensor(),
    # 흑백 이미지이기 때문에 1개의 채널을 정규화
    transforms.Normalize([0.5], [0.5])
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

])

# 데이터가 저장된 경로
data_dir = './semes_transfer'
print(os.path.join(data_dir, 'train'))

# 데이터가 저장된 경로에서 ImageFolder를 이용하여 이미지 데이터셋을 전처리한 후 로드(transforms_*==전처리 수행)
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), train_transform)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'val'), test_transform)

# DataLoader를 이용 / 데이터셋에서 미니배치(minibatch)를 추출 
# (batch_size==미니배치의 크기 / shuffle==데이터셋을 섞을지 여부 / num_workers==데이터셋을 불러올 때 사용할 프로세스 수)
train_loader = DataLoader(train_datasets, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_datasets, batch_size=128, shuffle=True, num_workers=4)

# 수행 결과를 출력
print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

# 학습된 클래스 이름과 수행 결과를 출력
class_names = train_datasets.classes
print('클래스:', class_names)

#### 전이학습 - 모델 불러오기

In [ ]:
# 모델 불러오기
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
CLASSIFICATION_MODEL_DIR = os.path.join(os.path.join(BASE_DIR, "models"), "classification_model.pth")
model = torch.load(CLASSIFICATION_MODEL_DIR)

# 불러온 네트워크 모델의 출력 뉴런 수를 저장
num_features = model.fc.in_features
# 새로운 Fully Connected 레이어 추가
model.fc = nn.Linear(num_features, 4)

# GPU를 사용하기 위해 모델을 CUDA 디바이스로 보냄
model.to(device)

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### 전이학습

In [ ]:
# 학습 epochs 설정
num_epochs = '설정값'

# epoch에 따른 손실 값과 정확도를 저장하는 리스트
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

# 최적 모델
best_loss = '현재 모델의 loss'
best_loss_epoch = 0

best_acc = '현재 모델의 정확도'
best_acc_epoch = 0

In [ ]:
# 현재시간 저장
start_time = time.time()

# 설정한 epochs 만큼 반복
for epoch in range(num_epochs):

    # Train --------------------------------------------------------------------
    # 모델을 학습모드로 설정
    model.train()
    # 현재까지 누적된 손실을 저장할 변수 초기화
    train_loss = 0
    # 현재까지 맞춘 이미지의 수를 저장할 변수 초기화
    train_correct = 0
    # 현재까지 학습한 이미지 수를 저장할 변수 초기화
    train_cnt = 0

    # 배치 단위로 나눈 학습 데이터 순회하며 불러와서
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        # 입력 이미지, 라벨 정보를 GPU를 사용하기 위해 to.device()사용
        inputs, labels = inputs.to(device), labels.to(device)
        # 학습 전, 이전 학습에서 계산된 gradient 값을 0으로 초기화
        optimizer.zero_grad()
        # 모델에 이미지 학습 (forward propagation이 이루어지며, 모델은 입력을 받아 출력값을 계산)
        outputs = model(inputs)
        # 모델이 예측한 출력값(outputs)과 실제 정답(labels)을 비교하여 손실 값을 계산
        loss = criterion(outputs, labels)
        # 손실 값의 gradient를 계산(backward propagation이 이루어지며, 손실 함수를 모델의 출력값으로 미분한 gradient 값을 계산)
        loss.backward()
        # optimization - 계산된 gradient 값을 이용하여 모델의 파라미터를 업데이트
        optimizer.step()

        # 현재 배치에서 계산된 손실 값을 train_loss에 더함
        train_loss += loss.item()
        # 출력값(outputs) 중에서 가장 큰 값(_)과 그 값이 존재하는 인덱스(predicted)를 반환
        _, predicted = outputs.max(1)
        # 배치에 포함되어 학습한 이미지의 수를 train_cnt에 더함
        train_cnt += labels.size(0)
        # 현재 배치에서 맞춘 개수를 train_correct에 더함
        train_correct += predicted.eq(labels).sum().item()
    
    # epoch 단위로 평균 손실 값과 정확도를 계산
    train_loss /= len(train_loader)
    train_acc = 100 * train_correct / train_cnt

    # 계산한 평균 손실 값과 정확도를 리스트에 추가
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)

    # Test ---------------------------------------------------------------------
    # 모델을 평가 모드로 설정
    model.eval()
    # 현재까지 누적된 손실을 저장할 변수 초기화
    test_loss = 0
    # 현재까지 맞춘 이미지의 수를 저장할 변수 초기화
    test_correct = 0
    # 현재까지 학습한 이미지 수를 저장할 변수 초기화
    test_cnt = 0

    # 모델의 gradient가 필요하지 않아 파라미터에 대한 업데이트를 수행하지 않으면서 forward propagation을 수행해 시간 단축
    with torch.no_grad():
         # 배치 단위로 나눈 테스트 데이터 순회하며 불러와서
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            # 입력 이미지, 라벨 정보를 GPU를 사용하기 위해 to.device()사용
            inputs, labels = inputs.to(device), labels.to(device)
            # 모델에 이미지 학습 (forward propagation이 이루어지며, 모델은 입력을 받아 출력값을 계산)
            outputs = model(inputs)
            # 모델이 예측한 출력값(outputs)과 실제 정답(labels)을 비교하여 손실 값을 계산
            loss = criterion(outputs, labels)

            # 현재 배치에서 계산된 손실 값을 test_loss에 더함
            test_loss += loss.item()
            # 출력값(outputs) 중에서 가장 큰 값(_)과 그 값이 존재하는 인덱스(predicted)를 반환
            _, predicted = outputs.max(1)
            # 배치에 포함되어 학습한 이미지의 수를 test_cnt에 더함
            test_cnt += labels.size(0)
            # 현재 배치에서 맞춘 개수를 test_correct에 더함
            test_correct += predicted.eq(labels).sum().item()
    
    # epoch 단위로 평균 손실 값과 정확도를 계산
    test_loss /= len(test_loader)
    test_acc = 100 * test_correct / test_cnt
    # 계산한 평균 손실 값과 정확도를 리스트에 추가
    test_loss_list.append(test_loss)
    test_acc_list.append(test_acc)

    # epoch마다 정확도와 손실률 출력
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.2f}%, Test Loss: {:.4f}, Test Acc: {:.2f}%, Time: {:.4f}s'
          .format(epoch + 1, num_epochs, train_loss, train_acc, test_loss, test_acc, time.time() - start_time))
    if test_loss < best_loss and test_acc < best_acc:
        best_loss = test_loss
        best_acc = test_acc
        best_epoch = epoch
        torch.save(model, 'model_new.pth')
    print('Best_Epoch : {}, Best_loss : {}, Best_acc : {}'.format(best_epoch, best_loss, best_acc))
